## Navigators
Navigators provide the ability to drill into the result set providing a summary of the distribution.  They are particularly useful when you are interested in gathering the domain of values for a particular Property.  Navigators can be used against a specific View, used in conjunction with either a query, a filter or both.  Navigators can be simple or very powerful, but provide a very useful way to capture results in logical buckets.  

In [1]:
import refinitiv.data as rd
from refinitiv.data.content import search
import pandas as pd
import datetime as dt
import dateutil.relativedelta

# Default session - desktop
rd.open_session(app_key='Your API Key here')

<refinitiv.data.session.Definition object at 0x2225ed3f400 {name='default'}>

In [2]:
pd.set_option('display.max_colwidth', 140)
rd.__version__

'1.0.0b9'

#### Example - Property values
The most basic usage of navigators is to determine the available values within a given Property.  For example, when searching for assets, we may want to narrow down the specific country an asset is traded in.  We can determine the available countries to work with.

In [3]:
# The following will list the countries assets are listed.  We can use this output to filter based on a specific country.
response = search.Definition(
    top = 0,
    navigators = "RCSCountryLeaf(buckets:20)"   # Show the top 20
).get_data()
response.data.raw["Navigators"]["RCSCountryLeaf"]["Buckets"]

[{'Label': 'United States', 'Count': 12921436},
 {'Label': 'Eurobond', 'Count': 4250714},
 {'Label': 'China (Mainland)', 'Count': 1282061},
 {'Label': 'Colombia', 'Count': 1063339},
 {'Label': 'South Korea', 'Count': 568176},
 {'Label': 'Germany', 'Count': 485007},
 {'Label': 'Canada', 'Count': 411919},
 {'Label': 'France', 'Count': 249568},
 {'Label': 'India', 'Count': 198446},
 {'Label': 'Japan', 'Count': 165052},
 {'Label': 'Italy', 'Count': 135127},
 {'Label': 'Chile', 'Count': 131491},
 {'Label': 'Australia', 'Count': 105528},
 {'Label': 'Brazil', 'Count': 67149},
 {'Label': 'Switzerland', 'Count': 66553},
 {'Label': 'Spain', 'Count': 62379},
 {'Label': 'Russia', 'Count': 59183},
 {'Label': 'Austria', 'Count': 58373},
 {'Label': 'Norway', 'Count': 57720},
 {'Label': 'United Kingdom', 'Count': 55412}]

#### Example - Vessels
If I want to perform a basic search for tankers and capture the Hull Types, I could start with the following basic query:

In [4]:
response=search.Definition(
    view = search.SearchViews.VESSEL_PHYSICAL_ASSETS,
    query = "tanker",
    select = "DocumentTitle, HullType",
    top = 10000
).get_data()
response.data.df

,DocumentTitle,HullType
0,"MOZAH, LNG Tanker, NAKILAT SHIPPING QATAR LTD, Gulf of Oman|Arabian Gulf Tanker Zone",Single Hull
1,"UMM SLAL, Liquefied Natural Gas; LNG Tanker, NAKILAT SHIPPING QATAR LTD, Arabia Sea|Arabian Gulf Tanker Zone",Single Hull
2,"BU SAMRA, Liquefied Natural Gas; LNG Tanker, NAKILAT SHIPPING QATAR LTD, South China Sea|Thailand Tanker Zone",Single Hull
3,"AL MAYEDA, LNG Tanker, STASCO SHIP MANAGEMENT, South China Sea|Thailand Tanker Zone",Single Hull
4,"MEKAINES, LNG Tanker, NAKILAT SHIPPING QATAR LTD, Gulf of Oman|Arabian Gulf Tanker Zone",Single Hull
...,...,...
9995,"TONG ZHOU JI 186, Other Tanker, Jiangsu|Shanghai Tanker Zone|Yangtze River|China (Mainland)",<NA>
9996,"HAI TAI 1996, Other Tanker, South China Sea|Canton Tanker Zone",<NA>
9997,"XIN HAI YUN HAO, Other Tanker, Zhejiang|East China Sea|Shanghai Tanker Zone|China (Mainland)",<NA>
9998,"PINGNANYONGJIA808, Other Tanker, South China Sea|Canton Tanker Zone",<NA>


Here, we can see a list of the different tankers and their Hull Types. Within this collection, while I can clearly see there are different Hull Types list, i.e. "Single", "Double", "NA", etc - I'm also seeing duplicates.  Instead of going through the effort to process the above result to pull out the unique list of Hull Types, I will instead use a Navigator - which was designed to do this work for me.

Using the above query, I will apply a Navigator by selecting the 'HullType' Property.  What this does is instruct search to bucket all unique Hull Types and summarize the total found for each. To capture the results, I will need to utilize the Content Layer API call.

In [5]:
response=search.Definition(
    view = search.SearchViews.VESSEL_PHYSICAL_ASSETS,
    query = "tanker",
    top = 0,       # I'm not interested in the hits, only details within the navigator response section
    navigators = "HullType"
).get_data()
response.data.raw["Navigators"]["HullType"]["Buckets"]

[{'Label': 'Double Hull', 'Count': 16699},
 {'Label': 'Single Hull', 'Count': 5432}]

#### Example - Listing of exchanges per asset category
In the following example, I'm going to list the top 3 asset types in Canada and for each, display the top 2 exchanges trading on those assets.

In [16]:
response=search.Definition(
    view = search.SearchViews.EQUITY_QUOTES,
    filter = "AssetState eq 'AC' and RCSExchangeCountryLeaf eq 'Canada'",    # Filter on all active (AC) assets within Canada
    top = 0,
    navigators = "RCSAssetCategoryLeaf(buckets:3, sub:ExchangeName(buckets:2))"  #  For the top 3 categories, show the top 2 exchanges
).get_data()

In [17]:
# Pretty display of the listing
rank = 1
for asset in response.data.raw["Navigators"]["RCSAssetCategoryLeaf"]["Buckets"]:
    print(f'{rank}. {asset["Label"]}')
    exchangeRank = 1
    for exchange in asset["ExchangeName"]["Buckets"]:
        print(f'\t{exchangeRank}. {exchange["Label"]} ({exchange["Count"]})')
        exchangeRank += 1
    rank += 1

1. Equity Cash Option
	1. Montreal Options Exchange (100072)
	2. Refinitiv (47872)
2. Ordinary Share
	1. Aequitas NEO-L (3505)
	2. Aequitas NEO-N (3504)
3. Equity Future Spread
	1. Montreal Exchange (5587)
	2. Montreal Options Exchange (17)


#### Example - Outstanding Government debt in the last 3 months.
In this example, I will be using some nested expressions to summarize categories within the buckets I collect.  Specifically, I'm looking for government debt ('govt') issued any time after 3-months ago using the following filter:
- "DBType eq 'govt' and IssueDate gt today-3months" (DBType identifies government debt)
    
Based on the result set, I want categorize or create buckets (top 5) based on the domicile of the issuer.  For each domicile, sum the outstanding debt and also include the maximum coupon rate for each one.
- RCSDomicileLeaf(buckets:5, desc:sum_FaceOutstandingUSD, calc:max_CouponRate)

In [18]:
# Get date N months ago
months = 3
date = dt.datetime.today() - dateutil.relativedelta.relativedelta(months=months)

In [19]:
# Retrieve the top 5 outstanding government debt in the last 3 months
response = search.Definition(
    view = search.SearchViews.GOV_CORP_INSTRUMENTS,
    filter = "DbType eq 'govt' and IssueDate gt " + date.strftime("%Y-%m-%d"),
    top = 0,
    navigators = "RCSDomicileLeaf(buckets:5, desc:sum_FaceOutstandingUSD, calc:max_CouponRate)"
).get_data()
response.data.raw["Navigators"]["RCSDomicileLeaf"]["Buckets"]

[{'Label': 'United States',
  'Count': 75,
  'max_CouponRate': 2.375,
  'sum_FaceOutstandingUSD': 2374169740400.0},
 {'Label': 'Japan',
  'Count': 56,
  'max_CouponRate': 0.7,
  'sum_FaceOutstandingUSD': 1118353989767.0},
 {'Label': 'China (Mainland)',
  'Count': 26,
  'max_CouponRate': 3.52,
  'sum_FaceOutstandingUSD': 196858815245.0},
 {'Label': 'Singapore',
  'Count': 42,
  'max_CouponRate': 0.786,
  'sum_FaceOutstandingUSD': 169983117847.0},
 {'Label': 'Hong Kong',
  'Count': 35,
  'max_CouponRate': 0.94,
  'sum_FaceOutstandingUSD': 119726393264.0}]